In [1]:
library(readxl)
library(ggplot2)
library(caret)
library(yardstick)
library(tibble)
library(Metrics)
library(e1071)
library(ROCR)
library(Hmisc)
library(MLmetrics)
library(mccr)
library(psych)
library(irr)
library(ROCR)
library(PRROC)
library(dplyr)
library(readxl)

Warning message:
"package 'readxl' was built under R version 4.3.3"
Warning message:
"package 'ggplot2' was built under R version 4.3.3"
Warning message:
"package 'caret' was built under R version 4.3.3"
Loading required package: lattice

Warning message:
"package 'lattice' was built under R version 4.3.3"
Warning message:
"package 'yardstick' was built under R version 4.3.3"

Attaching package: 'yardstick'


The following objects are masked from 'package:caret':

    precision, recall, sensitivity, specificity


Warning message:
"package 'Metrics' was built under R version 4.3.3"

Attaching package: 'Metrics'


The following objects are masked from 'package:yardstick':

    accuracy, mae, mape, mase, precision, recall, rmse, smape


The following objects are masked from 'package:caret':

    precision, recall


Warning message:
"package 'e1071' was built under R version 4.3.3"
Warning message:
"package 'Hmisc' was built under R version 4.3.3"

Attaching package: 'Hmisc'


The followin

In [2]:
data <- read_excel("C:/Users/SADEGHI/Desktop/US_MRI_Data.xlsx")
head(data)
y_true  <- data$y_true
y_pred <- data$y_pred

y_true,y_pred,y_pred_proba
<dbl>,<dbl>,<dbl>
3,3,0.17
4,4,0.44
5,5,0.30
4,4,0.36
5,5,0.17
4,4,0.47


In [3]:
num_classes <- length(unique(y_true))

In [4]:
# Calculate accuracy
accuracy <- sum(y_true == y_pred) / length(y_true)
conf_matrix <- table(y_true, y_pred)
class_support <- table(y_true)
tp <- sum(diag(conf_matrix))  # True Positives
fp <- sum(rowSums(conf_matrix)) - tp  # False Positives
fn <- sum(colSums(conf_matrix)) - tp  # False Negatives
precision_per_class <- diag(conf_matrix) / colSums(conf_matrix)
macro_precision <- mean(precision_per_class, na.rm = TRUE)
micro_precision <- tp / (tp + fp)
support <- rowSums(conf_matrix)
weighted_precision <- sum(precision_per_class * support) / sum(support)
recall_per_class <- diag(conf_matrix) / rowSums(conf_matrix)
macro_recall <- mean(recall_per_class, na.rm = TRUE)
micro_recall <- tp / (tp + fn)
weighted_recall <- sum(recall_per_class * support) / sum(support)
f1_per_class <- 2 * (precision_per_class * recall_per_class) / (precision_per_class + recall_per_class)
macro_f1 <- mean(f1_per_class, na.rm = TRUE)
micro_f1 <- tp / (tp + 0.5 * (fp + fn))
weighted_f1 <- sum(f1_per_class * support) / sum(support)
f_beta_score <- function(tp, fp, fn, beta = 0.5) {
  
  precision <- tp / (tp + fp)
  recall <- tp / (tp + fn)
  
  
  f_beta <- (1 + beta^2) * (precision * recall) / ((beta^2 * precision) + recall)
  
  return(f_beta)
}
f_beta <- f_beta_score(tp, fp, fn, beta = 0.5)
class_labels <- levels(factor(y_true))
calculate_mcc <- function(class_label, conf_matrix) {
  tp <- conf_matrix[class_label, class_label]  # True Positives for the class
  fn <- sum(conf_matrix[class_label, ]) - tp  # False Negatives for the class
  fp <- sum(conf_matrix[, class_label]) - tp  # False Positives for the class
  tn <- sum(conf_matrix) - tp - fp - fn  # True Negatives for the class
  
  # MCC formula
  numerator <- (tp * tn) - (fp * fn)
  denominator <- sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
  
  if (is.na(denominator) || denominator == 0) {
    return(0)
  } else {
    return(numerator / denominator)
  }
}
mcc_per_class <- sapply(class_labels, function(label) calculate_mcc(label, conf_matrix))
macro_mcc <- mean(mcc_per_class)
calculate_balanced_accuracy <- function(class_label, conf_matrix) {
  tp <- conf_matrix[class_label, class_label]
  fn <- sum(conf_matrix[class_label, ]) - tp
  fp <- sum(conf_matrix[, class_label]) - tp
  tn <- sum(conf_matrix) - (tp + fn + fp)

  TPR <- tp / (tp + fn)  # True Positive Rate (Sensitivity)
  TNR <- tn / (tn + fp)  # True Negative Rate (Specificity)
  
  balanced_accuracy <- (TPR + TNR) / 2
  
  return(balanced_accuracy)
}
balanced_accuracy_per_class <- sapply(class_labels, function(label) calculate_balanced_accuracy(label, conf_matrix))
macro_balanced_accuracy <- mean(balanced_accuracy_per_class)
TP <- diag(conf_matrix)
FN <- rowSums(conf_matrix) - TP
recall_per_class <- TP / (TP + FN)
n <- length(unique(y_true))
weighted_balanced_accuracy <- sum(recall_per_class) / n
TP <- diag(conf_matrix)
FP <- colSums(conf_matrix) - TP
FN <- rowSums(conf_matrix) - TP

jaccard_per_class <- TP / (TP + FP + FN)
macro_jaccard <- mean(jaccard_per_class, na.rm = TRUE)
micro_tp <- sum(TP)
micro_fp <- sum(FP)
micro_fn <- sum(FN)
micro_jaccard <- micro_tp / (micro_tp + micro_fp + micro_fn)
weighted_jaccard <- sum(jaccard_per_class * class_support) / sum(class_support)
results <- list(
  accuracy = sprintf("%.15f", accuracy),
  precision_per_class = setNames(as.list(sprintf("%.15f", precision_per_class)), class_labels),
  macro_precision = sprintf("%.15f", macro_precision),
  micro_precision = sprintf("%.15f", micro_precision),
  weighted_precision = sprintf("%.15f", weighted_precision),
  recall_per_class = setNames(as.list(sprintf("%.15f", recall_per_class)), class_labels),
  macro_recall = sprintf("%.15f", macro_recall),
  micro_recall = sprintf("%.15f", micro_recall),
  weighted_recall = sprintf("%.15f", weighted_recall),
  f1_per_class = setNames(as.list(sprintf("%.15f", f1_per_class)), class_labels),
  macro_f1 = sprintf("%.15f", macro_f1),
  micro_f1 = sprintf("%.15f", micro_f1),
  weighted_f1 = sprintf("%.15f", weighted_f1),
  f_beta_score = sprintf("%.15f", f_beta),
  mcc_per_class = setNames(as.list(sprintf("%.15f", mcc_per_class)), class_labels),
  macro_mcc = sprintf("%.15f", macro_mcc),
  balanced_accuracy_per_class = setNames(as.list(sprintf("%.15f", balanced_accuracy_per_class)), class_labels),
  macro_balanced_accuracy = sprintf("%.15f", macro_balanced_accuracy),
  weighted_balanced_accuracy = sprintf("%.15f", weighted_balanced_accuracy),
  jaccard_per_class = setNames(as.list(sprintf("%.15f", jaccard_per_class)), class_labels),
  macro_jaccard = sprintf("%.15f", macro_jaccard),
  micro_jaccard = sprintf("%.15f", micro_jaccard),
  weighted_jaccard = sprintf("%.15f", weighted_jaccard)
)
cat("Results:\n")
cat(sprintf("Accuracy: %s\n", results$accuracy))
cat("Precision per class:\n")
for (class in class_labels) {
  cat(sprintf("  Class %s: %s\n", class, results$precision_per_class[[class]]))
}
cat(sprintf("Macro Precision: %s\n", results$macro_precision))
cat(sprintf("Micro Precision: %s\n", results$micro_precision))
cat(sprintf("Weighted Precision: %s\n", results$weighted_precision))

cat("Recall per class:\n")
for (class in class_labels) {
  cat(sprintf("  Class %s: %s\n", class, results$recall_per_class[[class]]))
}
cat(sprintf("Macro Recall: %s\n", results$macro_recall))
cat(sprintf("Micro Recall: %s\n", results$micro_recall))
cat(sprintf("Weighted Recall: %s\n", results$weighted_recall))

cat("F1 Score per class:\n")
for (class in class_labels) {
  cat(sprintf("  Class %s: %s\n", class, results$f1_per_class[[class]]))
}
cat(sprintf("Macro F1 Score: %s\n", results$macro_f1))
cat(sprintf("Micro F1 Score: %s\n", results$micro_f1))
cat(sprintf("Weighted F1 Score: %s\n", results$weighted_f1))

cat(sprintf("F_beta Score (beta=0.5): %s\n", results$f_beta_score))

cat("MCC per class:\n")
for (class in class_labels) {
  cat(sprintf("  Class %s: %s\n", class, results$mcc_per_class[[class]]))
}
cat(sprintf("Macro MCC: %s\n", results$macro_mcc))

cat("Balanced Accuracy per class:\n")
for (class in class_labels) {
  cat(sprintf("  Class %s: %s\n", class, results$balanced_accuracy_per_class[[class]]))
}
cat(sprintf("Macro Balanced Accuracy: %s\n", results$macro_balanced_accuracy))
cat(sprintf("Weighted Balanced Accuracy: %s\n", results$weighted_balanced_accuracy))

cat("Jaccard Index per class:\n")
for (class in class_labels) {
  cat(sprintf("  Class %s: %s\n", class, results$jaccard_per_class[[class]]))
}
cat(sprintf("Macro Jaccard: %s\n", results$macro_jaccard))
cat(sprintf("Micro Jaccard: %s\n", results$micro_jaccard))
cat(sprintf("Weighted Jaccard: %s\n", results$weighted_jaccard))


Results:
Accuracy: 0.788135593220339
Precision per class:
  Class 3: 0.937500000000000
  Class 4: 0.600000000000000
  Class 5: 0.900000000000000
Macro Precision: 0.812500000000000
Micro Precision: 0.788135593220339
Weighted Precision: 0.829449152542373
Recall per class:
  Class 3: 0.900000000000000
  Class 4: 0.882352941176471
  Class 5: 0.529411764705882
Macro Recall: 0.770588235294118
Micro Recall: 0.788135593220339
Weighted Recall: 0.788135593220339
F1 Score per class:
  Class 3: 0.918367346938776
  Class 4: 0.714285714285714
  Class 5: 0.666666666666667
Macro F1 Score: 0.766439909297052
Micro F1 Score: 0.788135593220339
Weighted F1 Score: 0.787040239824743
F_beta Score (beta=0.5): 0.788135593220339
MCC per class:
  Class 3: 0.860961818027281
  Class 4: 0.590471938285626
  Class 5: 0.610323448812662
Macro MCC: 0.687252401708523
Balanced Accuracy per class:
  Class 3: 0.927941176470588
  Class 4: 0.822128851540616
  Class 5: 0.752801120448179
Macro Balanced Accuracy: 0.83429038281979

In [5]:
confMatrix <- confusionMatrix(factor(y_pred), factor(y_true))
cm_table <- confMatrix$table
class_labels <- levels(factor(y_true)) 
TP <- diag(cm_table)
FP <- colSums(cm_table) - TP
FN <- rowSums(cm_table) - TP
class_support <- table(factor(y_true))  
accuracy <- confMatrix$overall['Accuracy']
cat(sprintf("Accuracy (caret): %.15f\n", accuracy))
recall_per_class <- confMatrix$byClass[, "Sensitivity"]
macro_recall <- mean(recall_per_class, na.rm = TRUE)
weighted_recall <- sum(recall_per_class * (class_support / sum(class_support)))
micro_recall <- sum(TP) / (sum(TP) + sum(FN))

cat("\nRecall per class:\n")
for (i in 1:length(class_labels)) {
  cat(sprintf("Class %s: %.15f\n", class_labels[i], recall_per_class[i]))
}
cat(sprintf("Macro Recall (caret): %.15f\n", macro_recall))
cat(sprintf("Weighted Recall (caret): %.15f\n", weighted_recall))
cat(sprintf("Micro Recall (caret): %.15f\n", micro_recall))
precision_per_class <- confMatrix$byClass[, "Precision"]
macro_precision <- mean(precision_per_class, na.rm = TRUE)
micro_precision <- sum(TP) / (sum(TP) + sum(FP))
weighted_precision <- sum(precision_per_class * (class_support / sum(class_support)))

cat("\nPrecision per class:\n")
for (i in 1:length(class_labels)) {
  cat(sprintf("Class %s: %.15f\n", class_labels[i], precision_per_class[i]))
}
cat(sprintf("Macro Precision (caret): %.15f\n", macro_precision))
cat(sprintf("Micro Precision (caret): %.15f\n", micro_precision))
cat(sprintf("Weighted Precision (caret): %.15f\n", weighted_precision))
f1_per_class <- 2 * (precision_per_class * recall_per_class) / (precision_per_class + recall_per_class)
macro_f1 <- mean(f1_per_class, na.rm = TRUE)
weighted_f1 <- sum(f1_per_class * (class_support / sum(class_support)))
micro_f1 <- 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall)

cat("\nF1 Score per class:\n")
for (i in 1:length(class_labels)) {
  cat(sprintf("Class %s: %.15f\n", class_labels[i], f1_per_class[i]))
}
cat(sprintf("Macro F1 Score (caret): %.15f\n", macro_f1))
cat(sprintf("Weighted F1 Score (caret): %.15f\n", weighted_f1))
cat(sprintf("Micro F1 Score (caret): %.15f\n", micro_f1))
kappa_base <- confMatrix$overall['Kappa']
cat(sprintf("\nKappa (caret): %.15f\n", kappa_base))
tp_sum <- 0
tn_sum <- 0
fp_sum <- 0
fn_sum <- 0

for (i in 1:length(class_labels)) {
  tp <- cm_table[i, i]
  fn <- sum(cm_table[i, ]) - tp
  fp <- sum(cm_table[, i]) - tp
  tn <- sum(cm_table) - (tp + fn + fp)
  
  tp_sum <- tp_sum + tp
  tn_sum <- tn_sum + tn
  fp_sum <- fp_sum + fp
  fn_sum <- fn_sum + fn
}

numerator <- (tp_sum * tn_sum) - (fp_sum * fn_sum)
denominator <- sqrt((tp_sum + fp_sum) * (tp_sum + fn_sum) * (tn_sum + fp_sum) * (tn_sum + fn_sum))
mcc1 <- ifelse(denominator == 0, 0, numerator / denominator)  # Handle division by zero
cat(sprintf("\nMCC (caret): %.15f\n", mcc1))
balanced_acc <- mean(confMatrix$byClass[,"Balanced Accuracy"])
cat(sprintf("\nBalanced Accuracy (caret): %.15f\n", balanced_acc))
jaccard_per_class <- TP / (TP + FP + FN)
macro_jaccard <- mean(jaccard_per_class, na.rm = TRUE)
micro_tp <- sum(TP)
micro_fp <- sum(FP)
micro_fn <- sum(FN)
micro_jaccard <- micro_tp / (micro_tp + micro_fp + micro_fn)
weighted_jaccard <- sum(jaccard_per_class * (class_support / sum(class_support)))

cat("\nJaccard Index per class:\n")
for (i in 1:length(class_labels)) {
  cat(sprintf("Class %s: %.15f\n", class_labels[i], jaccard_per_class[i]))
}
cat(sprintf("Macro Jaccard Index (caret): %.15f\n", macro_jaccard))
cat(sprintf("Micro Jaccard Index (caret): %.15f\n", micro_jaccard))
cat(sprintf("Weighted Jaccard Index (caret): %.15f\n", weighted_jaccard))


Accuracy (caret): 0.788135593220339

Recall per class:
Class 3: 0.900000000000000
Class 4: 0.882352941176471
Class 5: 0.529411764705882
Macro Recall (caret): 0.770588235294118
Weighted Recall (caret): 0.788135593220339
Micro Recall (caret): 0.788135593220339

Precision per class:
Class 3: 0.937500000000000
Class 4: 0.600000000000000
Class 5: 0.900000000000000
Macro Precision (caret): 0.812500000000000
Micro Precision (caret): 0.788135593220339
Weighted Precision (caret): 0.829449152542373

F1 Score per class:
Class 3: 0.918367346938776
Class 4: 0.714285714285714
Class 5: 0.666666666666667
Macro F1 Score (caret): 0.766439909297052
Weighted F1 Score (caret): 0.787040239824743
Micro F1 Score (caret): 0.788135593220339

Kappa (caret): 0.677384076990376

MCC (caret): 0.682203389830508

Balanced Accuracy (caret): 0.834290382819795

Jaccard Index per class:
Class 3: 0.849056603773585
Class 4: 0.555555555555556
Class 5: 0.500000000000000
Macro Jaccard Index (caret): 0.634870719776380
Micro Jac

In [6]:
accuracy <- Accuracy(y_pred, y_true)
cat(sprintf("Accuracy (MLmetrics): %.15f\n", accuracy))
conf_table <- table(y_true, y_pred)
TP <- diag(conf_table)
FN <- rowSums(conf_table) - TP
class_support <- table(y_true)  
recall_per_class <- sapply(levels(factor(y_true)), function(class) {
  Recall(y_pred = y_pred, y_true = y_true, positive = class)
})

macro_recall <- mean(recall_per_class, na.rm = TRUE)
micro_recall <- sum(TP) / (sum(TP) + sum(FN))
weighted_recall <- sum(recall_per_class * (class_support / sum(class_support)))

cat("\nClass-based Recall (MLmetrics):\n")
for (i in 1:length(recall_per_class)) {
  cat(sprintf("Class %s: %.15f\n", levels(factor(y_true))[i], recall_per_class[i]))
}
cat(sprintf("Macro Recall (MLmetrics): %.15f\n", macro_recall))
cat(sprintf("Weighted Recall (MLmetrics): %.15f\n", weighted_recall))
cat(sprintf("Micro Recall (MLmetrics): %.15f\n", micro_recall))
actual <- factor(y_true, levels = unique(c(y_true, y_pred)))
predicted <- factor(y_pred, levels = unique(c(y_true, y_pred)))

precision_per_class <- sapply(levels(actual), function(class) {
  Precision(y_pred = predicted, y_true = actual, positive = class)
})

macro_precision <- mean(precision_per_class, na.rm = TRUE)
micro_precision <- sum(TP) / (sum(TP) + sum(FP))  
FP <- colSums(conf_table) - TP
weighted_precision <- sum(precision_per_class * (class_support / sum(class_support)))

cat("\nClass-based Precision (MLmetrics):\n")
for (i in 1:length(precision_per_class)) {
  cat(sprintf("Class %s: %.15f\n", levels(factor(y_true))[i], precision_per_class[i]))
}
cat(sprintf("Macro Precision (MLmetrics): %.15f\n", macro_precision))
cat(sprintf("Weighted Precision (MLmetrics): %.15f\n", weighted_precision))
cat(sprintf("Micro Precision (MLmetrics): %.15f\n", micro_precision))
f1_scores_per_class <- sapply(levels(factor(y_true)), function(cls) {
  cls_numeric <- as.numeric(cls)
  y_true_binary <- as.numeric(y_true == cls_numeric)
  y_pred_binary <- as.numeric(y_pred == cls_numeric)
  F1_Score(y_true_binary, y_pred_binary)
})

macro_f1 <- mean(f1_scores_per_class, na.rm = TRUE)
weighted_f1 <- sum(f1_scores_per_class * (class_support / sum(class_support)))
micro_f1 <- 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall)

cat("\nClass-based F1 Score (MLmetrics):\n")
for (i in 1:length(f1_scores_per_class)) {
  cat(sprintf("Class %s: %.15f\n", levels(factor(y_true))[i], f1_scores_per_class[i]))
}
cat(sprintf("Macro F1 Score (MLmetrics): %.15f\n", macro_f1))
cat(sprintf("Weighted F1 Score (MLmetrics): %.15f\n", weighted_f1))
cat(sprintf("Micro F1 Score (MLmetrics): %.15f\n", micro_f1))


Accuracy (MLmetrics): 0.788135593220339

Class-based Recall (MLmetrics):
Class 3: 0.900000000000000
Class 4: 0.882352941176471
Class 5: 0.529411764705882
Macro Recall (MLmetrics): 0.770588235294118
Weighted Recall (MLmetrics): 0.788135593220339
Micro Recall (MLmetrics): 0.788135593220339

Class-based Precision (MLmetrics):
Class 3: 0.937500000000000
Class 4: 0.600000000000000
Class 5: 0.900000000000000
Macro Precision (MLmetrics): 0.812500000000000
Weighted Precision (MLmetrics): 0.829449152542373
Micro Precision (MLmetrics): 0.788135593220339

Class-based F1 Score (MLmetrics):
Class 3: 0.942028985507246
Class 4: 0.842105263157895
Class 5: 0.901098901098901
Macro F1 Score (MLmetrics): 0.895077716588014
Weighted F1 Score (MLmetrics): 0.901443990339774
Micro F1 Score (MLmetrics): 0.788135593220339


In [7]:
data1 <- data.frame(actual = factor(y_true), predicted = factor(y_pred))
results <- data1 %>%
  metrics(truth = actual, estimate = predicted)
accuracy_result <- results %>% filter(.metric == "accuracy")
sprintf("Accuracy (yardstick): %.15f", accuracy_result$.estimate)
precision_result <- results %>% filter(.metric == "precision")
sprintf("Precision (yardstick): %.15f", precision_result$.estimate)
recall_result <- results %>% filter(.metric == "recall")
sprintf("Recall (yardstick): %.15f", recall_result$.estimate)
f1_result <- results %>% filter(.metric == "f_meas")
sprintf("F1 Score (yardstick): %.15f", f1_result$.estimate)
kappa_result <- data1 %>%
  kappa(truth = actual, estimate = predicted)
sprintf("Kappa (yardstick): %.15f", kappa_result$.estimate)
bal_acc_result <- data1 %>%
  bal_accuracy(truth = actual, estimate = predicted)
sprintf("Balanced Accuracy (yardstick): %.15f", bal_acc_result$.estimate)
fbeta_macro_result <- data1 %>%
  f_meas(truth = actual, estimate = predicted, beta = 0.5, estimator = "macro")
sprintf("Macro F-beta Score (yardstick): %.15f", fbeta_macro_result$.estimate)

fbeta_micro_result <- data1 %>%
  f_meas(truth = actual, estimate = predicted, beta = 0.5, estimator = "micro")
sprintf("Micro F-beta Score (yardstick): %.15f", fbeta_micro_result$.estimate)
fbeta_weighted_result <- data1 %>%
  f_meas(truth = actual, estimate = predicted, beta = 0.5, estimator = "macro_weighted")
sprintf("Weighted F-beta Score (yardstick): %.15f", fbeta_weighted_result$.estimate)


[1] "Accuracy (yardstick): 0.788135593220339"

character(0)

character(0)

character(0)

character(0)

[1] "Balanced Accuracy (yardstick): 0.834290382819795"

[1] "Macro F-beta Score (yardstick): 0.786750463783957"

[1] "Micro F-beta Score (yardstick): 0.788135593220339"

[1] "Weighted F-beta Score (yardstick): 0.806140511557753"

In [8]:
### e1071 Library ###
kappa_e1071 <- kappa(conf_matrix)
sprintf("Cohen's Kappa (e1071): %.15f", kappa_e1071$coef)

[1] "Cohen's Kappa (e1071): 0.677384076990376"

In [9]:
### Psych Library ###
kappa_psych <- cohen.kappa(conf_matrix)
sprintf("Cohen's Kappa (psych): %.15f", kappa_psych$kappa)

[1] "Cohen's Kappa (psych): 0.677384076990376"

In [10]:
### IRR Library ###
kappa_irr <- kappa2(data1)
sprintf("Cohen's Kappa (irr): %.15f", kappa_irr$value)

[1] "Cohen's Kappa (irr): 0.677384076990376"